In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff




In [2]:
yandex_world = pd.read_csv('yandex_world.csv')
yandex_world.rename(columns = {'Дата':'Date','Страна':'Country','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)
for i in range(yandex_world.shape[0]):
    a= yandex_world.Date[i].split('.')
    yandex_world.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))

yandex_world.tail(20)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
41120,Япония,2020-08-09,48782,32867,1047,1440,1619,5
41121,Япония,2020-08-10,49617,33450,1052,835,583,5
41122,Япония,2020-08-11,50302,34136,1058,685,686,6
41123,Япония,2020-08-12,51288,34969,1066,986,833,8
41124,Япония,2020-08-13,52471,36191,1073,1183,1222,7
41125,Япония,2020-08-14,53818,35498,1080,1347,-693,7
41126,Япония,2020-08-15,55051,39439,1093,1233,3941,13
41127,Япония,2020-08-16,56074,40560,1103,1023,1121,10
41128,Япония,2020-08-17,56717,41591,1112,643,1031,9
41129,Япония,2020-08-18,57636,42965,1135,919,1374,23


In [3]:
df3_temp = yandex_world.groupby('Date').sum()
df3_temp['Country']='Весь мир'

y5= pd.concat([yandex_world,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_world = y5
yandex_world.tail(10)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
210,Весь мир,2020-08-19,22408212,14333914,787684,273533,217463,6753
211,Весь мир,2020-08-20,22675329,14541573,793708,267117,207659,6024
212,Весь мир,2020-08-21,22946050,14712252,799255,270721,170679,5547
213,Весь мир,2020-08-22,23200536,14911319,804419,254486,199067,5164
214,Весь мир,2020-08-23,23417487,15137203,808680,216951,225884,4261
215,Весь мир,2020-08-24,23643821,15337252,813018,226334,200049,4338
216,Весь мир,2020-08-25,23885837,15570191,819416,242016,232939,6398
217,Весь мир,2020-08-26,24173588,15794441,825695,287751,224250,6279
218,Весь мир,2020-08-27,24452689,15997281,831586,279101,202840,5891
219,Весь мир,2020-08-28,24734448,16198165,837124,281759,200884,5538


In [4]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_world.tail(1).Date)[0]

print(yandex_world.iloc[-1,0])
print('Выбран день: ',myday)

Весь мир
Выбран день:  2020-08-28


In [5]:
X = pd.DataFrame()
country_dict = {}
yandex_world['Rt'] = 1
yandex_world['Rt2'] = 1
yandex_world['Rate_conf'] = 0
yandex_world['Rate_ill'] = 0

temp_df = yandex_world.groupby(['Country','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Country'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    country_dict[i]=j
    
print(country_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Австралия'), (1, 'Австрия'), (2, 'Азербайджан'), (3, 'Албания'), (4, 'Алжир'), (5, 'Ангола'), (6, 'Андорра'), (7, 'Антигуа и Барбуда'), (8, 'Аргентина'), (9, 'Армения'), (10, 'Афганистан'), (11, 'Багамские острова'), (12, 'Бангладеш'), (13, 'Барбадос'), (14, 'Бахрейн'), (15, 'Беларусь'), (16, 'Белиз'), (17, 'Бельгия'), (18, 'Бенин'), (19, 'Болгария'), (20, 'Боливия'), (21, 'Босния и Герцеговина'), (22, 'Ботсвана'), (23, 'Бразилия'), (24, 'Бруней'), (25, 'Буркина-Фасо'), (26, 'Бурунди'), (27, 'Бутан'), (28, 'Великобритания'), (29, 'Венгрия'), (30, 'Венесуэла'), (31, 'Весь мир'), (32, 'Восточный Тимор'), (33, 'Вьетнам'), (34, 'Габон'), (35, 'Гаити'), (36, 'Гайана'), (37, 'Гамбия'), (38, 'Гана'), (39, 'Гватемала'), (40, 'Гвинея'), (41, 'Гвинея-Бисау'), (42, 'Германия'), (43, 'Гондурас'), (44, 'Гренада'), (45, 'Греция'), (46, 'Грузия'), (47, 'ДР Конго'), (48, 'Дания'), (49, 'Джибути'), (50, 'Доминика'), (51, 'Доминиканская Республика'), (52, 'Египет'), (53, 'Замбия'), (54,

,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths,Rt,Rt2,Rate_conf,Rate_ill,Country,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
41355,2020-08-24,63158,49089,1201,500,770,13,0.852,1.067,0,0,Япония,12868,920.142857,967.214286,13401.571429,14254.500000,-242,-283
41356,2020-08-25,63888,50092,1219,730,1003,18,0.694,1.061,0,0,Япония,12577,893.142857,970.428571,13264.571429,14073.714286,230,-291
41357,2020-08-26,64779,51407,1230,891,1315,11,0.669,1.054,0,0,Япония,12142,864.428571,963.642857,13028.571429,13851.500000,161,-435
41358,2020-08-27,65653,52542,1241,874,1135,11,0.762,1.049,0,0,Япония,11870,821.857143,941.571429,12776.714286,13613.142857,-17,-272
41359,2020-08-28,66499,53691,1251,846,1149,10,1.025,1.049,0,0,Япония,11557,792.857143,905.785714,12497.714286,13207.214286,-28,-313


In [6]:
# Добавляем колонки расчитанные сколько доля их в процентах от суммарного показателя в мире
mask = X.Country == 'Весь мир'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
world_date = list(temp_df.index)
world_conf = list(temp_df.Confirmed)
world_ill = list(temp_df.Remaining_ill)
world_dict_conf = dict(zip(world_date,world_conf))
world_dict_ill = dict(zip(world_date,world_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/world_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/world_dict_ill.get(X.loc[i,'Date'])


Confirmed  Remaining_ill
Date                                
2020-08-27   24452689        7623822
2020-08-28   24734448        7699159
41360
(41360, 19)
2020-01-22


In [7]:
# Проверка расчетов процентных показателей
mask = (X.Date>=datetime.date(2020,6,13))&(X.Country.isin(['Весь мир','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Country','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Country,Rate_conf,Rate_ill
28360,2020-08-09,885718,Россия,4.460080,2.547503
28361,2020-08-10,890799,Россия,4.434863,2.551447
28362,2020-08-11,895691,Россия,4.403354,2.547538
28363,2020-08-12,900745,Россия,4.368694,2.506108
28364,2020-08-13,905762,Россия,4.332935,2.450190
28365,2020-08-14,910778,Россия,4.294406,2.424097
28366,2020-08-15,915808,Россия,4.268143,2.378498
28367,2020-08-16,920719,Россия,4.248941,2.403878
28368,2020-08-17,925558,Россия,4.230451,2.428981
28369,2020-08-18,930276,Россия,4.202799,2.395853


In [8]:
X.to_csv('W.csv',index=False)